# The Data
You will be working with a heavily modified version of the Superheroes dataset from Kaggle.

The dataset includes two csv's:

[superhero_info.csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vS1ZstYLwFgwhZnqDsPjtnlHYhJp_cmW55J8JD5mym0seRsaem3px7QBtuFF0LiI7z1PLCkVKAkdO7J/pub?output=csv):
Contains Name, Publisher, Demographic Info, and Body measurements.

[superhero_powers.csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vSzdWOBaXOoz52vPmCFV5idNlDBohLY1Lsbc1IfZIZQ7cV_aNB2wYBfhF49uE1TaO1B5MQCGWiNrFfd/pub?output=csv):
Contains Hero name and list of powers


In [1]:
# standard imports
import pandas as pd
import os
import json

In [2]:
#load both files

info = pd.read_csv("Data/superhero_info - superhero_info.csv")
powers = pd.read_csv("Data/superhero_powers - superhero_powers.csv")

print(info.info())
print('\n')
print(powers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB
None


In [3]:
info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [4]:
powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


# The Task

## Clean the files and combine them into one final DataFrame.



### Separate Hero and Publisher

In [5]:
# split hero and publisher
info[['Hero', 'Publisher']] = info['Hero|Publisher'].str.split('|', expand = True)
#drop original col
info = info.drop(columns = ["Hero|Publisher"])
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


### Decomposing Measurements column from info df

In [6]:
# height and weight
measurement = info.loc[0, "Measurements"]
print(type(measurement))
measurement

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [7]:
# lets convert str to dict 
## first change to ""
info['Measurements'] = info["Measurements"].str.replace("'",'"') 

In [8]:
## use .apply with json.loads
info['Measurements'] = info['Measurements'].apply(json.loads)
measurement = info.loc[0, 'Measurements']
print(type(measurement))

<class 'dict'>


In [9]:
# unpacck dict to df with new cols
height_and_weight = info["Measurements"].apply(pd.Series)

# concatenate new cols to original info df
info = pd.concat((info, height_and_weight), axis = 1)

# drop Measurements col
info = info.drop(columns = ["Measurements"])

In [10]:
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [11]:
# remove cm and kg inside height and weight cell
## make a list of all characters to replace
remove = [' cm', ' kg']

## run a loop to replace all of the characters in the list at once
for char in remove:
    info['Height'] = info["Height"].str.replace(char, '', regex = False)
    info['Weight'] = info["Weight"].str.replace(char, '', regex = False)

print(info.info())
info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      463 non-null    object
 1   Race        463 non-null    object
 2   Alignment   463 non-null    object
 3   Hair color  463 non-null    object
 4   Eye color   463 non-null    object
 5   Skin color  463 non-null    object
 6   Hero        463 non-null    object
 7   Publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB
None


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [12]:
# convert height and weight cols to numercial
info['Height'] = info['Height'].astype(float)
info['Weight'] = info['Weight'].astype(float)

info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


### Decomposing Powers column from powers df

In [13]:
power = powers.loc[0, "Powers"]
type(power)

str

In [14]:
# split string on comma
powers['powers_split'] = powers['Powers'].str.split(",")
powers['powers_split'].head()

0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: powers_split, dtype: object

In [15]:
# obtain unique values for Powers with .explode method
exploded = powers.explode('powers_split')
exploded.head()

,hero_names,Powers,powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing


In [16]:
# obtain unique names of cols to create one-hot-encode cols
cols_to_make = exploded['powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [17]:
# create new col for eacch unique superpower and populate cell with true or false

for col in cols_to_make:
    powers[col]=powers['Powers'].str.contains(col)
    
powers.head()

/var/folders/dj/tq7kx_ys6m161zqrzvh_73q80000gn/T/ipykernel_5202/3033736901.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col]=powers['Powers'].str.contains(col)
/var/folders/dj/tq7kx_ys6m161zqrzvh_73q80000gn/T/ipykernel_5202/3033736901.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col]=powers['Powers'].str.contains(col)
/var/folders/dj/tq7kx_ys6m161zqrzvh_73q80000gn/T/ipykernel_5202/3033736901.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

,hero_names,Powers,powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
# prepare a new df to combine with info
powers=powers.drop(columns=['Powers', 'powers_split'])
powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Concatenate info and power df to one final df

In [19]:
# use Hero from info and hero_names in powers to merge to a nes df
df_final = pd.merge(info, powers, left_on='Hero', right_on='hero_names')
df_final.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


In [20]:
# re-order final df for better presentation

first_cols = ['Hero', 'Publisher', 'Gender', 'Height', 'Weight']
last_cols = [col for col in df_final.columns if col not in first_cols]

df_final = df_final[first_cols+last_cols]
df_final

,Hero,Publisher,Gender,Height,Weight,Race,Alignment,Hair color,Eye color,Skin color,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Marvel Comics,Male,203.0,441.0,Human,good,No Hair,yellow,Unknown,...,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Dark Horse Comics,Male,191.0,65.0,Icthyo Sapien,good,No Hair,blue,blue,...,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,DC Comics,Male,185.0,90.0,Ungaran,good,No Hair,blue,red,...,False,False,False,False,False,False,False,False,False,False
3,Abomination,Marvel Comics,Male,203.0,441.0,Human / Radiation,bad,No Hair,green,Unknown,...,False,False,False,False,False,False,False,False,False,False
4,Absorbing Man,Marvel Comics,Male,193.0,122.0,Human,bad,No Hair,blue,Unknown,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Yellowjacket,Marvel Comics,Male,183.0,83.0,Human,good,Blond,blue,Unknown,...,False,False,False,False,False,False,False,False,False,False
459,Yellowjacket II,Marvel Comics,Female,165.0,52.0,Human,good,Strawberry Blond,blue,Unknown,...,False,False,False,False,False,False,False,False,False,False
460,Yoda,George Lucas,Male,66.0,17.0,Yoda's species,good,White,brown,green,...,False,False,False,False,False,False,False,False,False,False
461,Zatanna,DC Comics,Female,170.0,57.0,Human,good,Black,blue,Unknown,...,True,False,False,False,False,False,False,False,False,False


In [21]:
# optimize the name of height and weight col
df_final.columns = ['Height (cm)' if x=='Height' else 'Weight (kg)' if x=='Weight' else
                    x for x in df_final.columns]

print(df_final.info())
df_final.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Columns: 178 entries, Hero to Changing Armor
dtypes: bool(167), float64(2), object(9)
memory usage: 118.9+ KB
None


,Hero,Publisher,Gender,Height (cm),Weight (kg),Race,Alignment,Hair color,Eye color,Skin color,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Marvel Comics,Male,203.0,441.0,Human,good,No Hair,yellow,Unknown,...,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Dark Horse Comics,Male,191.0,65.0,Icthyo Sapien,good,No Hair,blue,blue,...,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,DC Comics,Male,185.0,90.0,Ungaran,good,No Hair,blue,red,...,False,False,False,False,False,False,False,False,False,False
3,Abomination,Marvel Comics,Male,203.0,441.0,Human / Radiation,bad,No Hair,green,Unknown,...,False,False,False,False,False,False,False,False,False,False
4,Absorbing Man,Marvel Comics,Male,193.0,122.0,Human,bad,No Hair,blue,Unknown,...,False,False,False,False,False,False,False,False,False,False


## Use your combined DataFrame to answer the following questions.



### Compare the average weight of super powers who have Super Speed to those who do not.


In [22]:
df_final.groupby('Super Speed')['Weight (kg)'].mean().round(2)

Super Speed
False    101.77
True     129.40
Name: Weight (kg), dtype: float64

### What is the average height of heroes for each publisher?

In [23]:
df_final.groupby('Publisher')['Height (cm)'].mean().round(2).sort_values(ascending=False)

Publisher
Image Comics         211.00
Marvel Comics        191.55
DC Comics            181.92
Star Trek            181.50
Team Epic TV         180.75
Unknown              178.00
Dark Horse Comics    176.91
Shueisha             171.50
George Lucas         159.60
Name: Height (cm), dtype: float64

In [24]:
# save df_final as csv.gz

df_final.to_csv('Data/superhero.csv.gz',compression='gzip',index=False)

#veriy file creation
FOLDER = "Data/"
os.listdir(FOLDER)

['superhero_info - superhero_info.csv',
 'superhero_powers - superhero_powers.csv',
 'superhero.csv.gz']